In [12]:
from open3d import *
import numpy as np
import copy

In [2]:
def draw_registration_result_original_color(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    draw_geometries([source_temp, target])

In [4]:
#Load two point clouds and show initial pose
source = read_point_cloud("TestData/ColoredICP/frag_115.ply")
target = read_point_cloud("TestData/ColoredICP/frag_116.ply")
    
#draw initial alignment
current_transformation = np.identity(4)
draw_registration_result_original_color(source, target, current_transformation)

In [13]:
# point to plane ICP
current_transformation = np.identity(4);
print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
result_icp = registration_icp(source, target, 0.02,current_transformation, TransformationEstimationPointToPlane())
print(result_icp)
draw_registration_result_original_color(source, target, result_icp.transformation)

2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
RegistrationResult with fitness = 0.974582, inlier_rmse = 0.004220, and correspondence_set size of 62729
Access transformation to get result.


Colored pointcloud registration.  

In [14]:
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = np.identity(4)
print("3. Colored point cloud registration")

for scale in range(3):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter,radius,scale])
    
    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = voxel_down_sample(source, radius)
    target_down = voxel_down_sample(target, radius)
    
    print("3-2. Estimate normal.")
    estimate_normals(source_down, KDTreeSearchParamHybrid(
                radius = radius * 2, max_nn = 30))
    estimate_normals(target_down, KDTreeSearchParamHybrid(
                radius = radius * 2, max_nn = 30))    
    
    print("3-3. Applying colored point cloud registration")
    result_icp = registration_colored_icp(source_down, target_down,
                radius, current_transformation,
                ICPConvergenceCriteria(relative_fitness = 1e-6,
                relative_rmse = 1e-6, max_iteration = iter))
    current_transformation = result_icp.transformation 
    draw_registration_result_original_color(source, target, result_icp.transformation)

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
[30, 0.02, 1]
3-1. Downsample with a voxel size 0.02
3-2. Estimate normal.
3-3. Applying colored point cloud registration
[14, 0.01, 2]
3-1. Downsample with a voxel size 0.01
3-2. Estimate normal.
3-3. Applying colored point cloud registration
